In [ ]:
#applicato a 6 file totali (invece di 35)

In [ ]:
final_neurons=[]
from NewLibraryENGcopia import *
import glob
#final=[]
list_dir=glob.glob("/Users/Gaia_1/Desktop/allh5filestutto/*")
output_path='/Users/Gaia_1/Desktop/tesi/Data after SS'
for file in tqdm(list_dir):
    target=1
    stim=0
    if 'health' in file:
        target=0
    if 'stimulation' in file:
        stim=1
    file_name = file.split("/")[-1]
    print(file_name,':','target',target,'stimulation',stim)
    neurons=spike_sorting(file,output_path)
    #df=poiproc(neurons,target,stim)
    #final.append(df)
    final_neurons.append(neurons)

In [ ]:
from NewLibraryENGcopia import *
import glob
final=[]
output_path='/Users/Gaia_1/Desktop/tesi/Data after SS'
list_dir=glob.glob(output_path+"/*")
for file in tqdm(list_dir):
    target=1
    stim=0
    if 'health' in file:
        target=0
    if 'stimulation' in file:
        stim=1
    list_neurons = np.genfromtxt(file, delimiter=',')
    df=poiproc(list_neurons,target,stim)
    final.append(df)

In [ ]:
df=this_poiproc(neurons,0,0)

In [ ]:
from NewLibraryENGcopia import *
import glob
final=[]
list_dir=glob.glob("/Users/Gaia_1/Desktop/tesi/Data after SS/*")
for file in list_dir:
    target=1
    stim=0
    if 'health' in file:
        target=0
    if 'stimulation' in file:
        stim=1
    file_name = file.split("/")[-1]
    print(file_name,':','target',target,'stimulation',stim)
    neurons = np.genfromtxt(file, delimiter=',')
    df=poiproc(neurons,target,stim)
    final.append(df)


In [ ]:
from NewLibraryENGcopia import *

#POPULATION 1 (HEALTHY):
name_data_st = '2019-01-23T16-06-32Pop1 healthy cortical .h5'
name_data_BL = '2019-01-23T11-19-05PoP1 healthy cortical .h5'
name_data_24 = '2019-01-24T16-00-33Pop1 24hour after.h5'
complete_string='/Users/Gaia_1/Desktop/allh5files/healthy/healthy_baseline/'+name_data_BL
#complete_string='/Users/Gaia_1/Desktop/allh5files/healthy/healthy_stimulation/'+name_data_st
#complete_string='/Users/Gaia_1/Desktop/allh5files/healthy/healthy 24hrs later/'+name_data_24

In [ ]:
from NewLibraryENGcopia import *
import glob
final=[]
list_dir=glob.glob("/Users/Gaia_1/Desktop/allh5filestutto/*")
for files in list_dir:
    print(files)
file=list_dir[1]
print('chosen:',file)

In [ ]:

target=1
stim=0
#healthy and LKRR2
if 'health' in file:
    target=0
if 'stimulation' in file:
    stim=1
#all files
file_name = file.split("/")[-1]
print(file_name,':','target',target,'stimulation',stim)
neurons=spike_sorting(file_name,file,1,4,1.5)
df=poiproc(neurons,target,stim)
final.append(df)
concatenated_dataframes = [elem.T for elem in tqdm(final)]
final_result = pd.concat(concatenated_dataframes, axis=0)
file_path = '/Users/Gaia_1/Desktop/tesi/Data after PP/'+file_name+'.csv'
final_result.to_csv(file_path, index=False)

In [ ]:

target=1
stim=0
#healthy and LKRR2
if 'health' in file:
    target=0
if 'stimulation' in file:
    stim=1
#all files
file_name = file.split("/")[-1]
print(file_name,':','target',target,'stimulation',stim)
neurons=spike_sorting(file_name,file,1,4,1.5)
df=poiproc(neurons,target,stim)
final.append(df)
concatenated_dataframes = [elem.T for elem in tqdm(final)]
final_result = pd.concat(concatenated_dataframes, axis=0)
file_path = '/Users/Gaia_1/Desktop/tesi/Data after PP/'+file_name+'.csv'
final_result.to_csv(file_path, index=False)


In [ ]:
len(neurons[4:])

In [ ]:
import arviz as az

df=this_poiproc(neurons[9:],target,stim)

In [ ]:
def this_poiproc(neurons,target,stim):
    from scipy.stats import ks_2samp
    dataframe = pd.DataFrame()
    counter=0
    list_neurons = neurons
    counter=0
    print('Original number of neurons: ',len(list_neurons))
    for neuron in tqdm(list_neurons):
        neuron=neuron[neuron>0*10000]
        #neuron=neuron[neuron>100*10000]
        neuron=neuron[neuron<200*10000]
        print('  Neuron with ',neuron.shape[0],'spikes')
        if neuron.shape[0]>1000:
            
            counter+=1
        else:
            print('    Excluded neuron with n spikes = ',neuron.shape[0])
            continue
        
        ISI_healthy = np.diff(neuron)/10000
        map_estimate,trace,ppc_trace = this_Bayesian_mixture_model(ISI_healthy)
        map_estimate['Target']=target
        map_estimate['Stimulation']=stim
        df = pd.DataFrame.from_dict(map_estimate,orient='index')
        dataframe = pd.concat([dataframe,df],axis = 1)
        lista_samples=[]
        for i in list(ppc_trace['posterior_predictive']['like'].values):
            lista_samples.extend(i)
        #ks_2samp(lista_samples,ISI_healthy,mode = 'asymp')
        #print('ks: ',ks_2amp)
        ks_statistic, p_value = ks_2samp(np.concatenate(lista_samples), ISI_healthy, mode='asymp')
        if p_value>=0.69:
            print('p value high enough')
            az.plot_trace(trace)
        print('ks_statistic:', ks_statistic,'p_value:', p_value)
        
    print('Final number of neurons: ',counter)
    print('Target = ',target)
    return dataframe


In [ ]:
def this_Bayesian_mixture_model(ISI_data):
    with pm.Model() as model:
        ##### WALD DISTRIBUTION (INVERSE GAUSSIAN)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


        mu2 = pm.Uniform('mu2',lower=0,upper=0.2)
        sigma2 = pm.Uniform('sigma2',lower=0.0001,upper=0.5)
        obs2 = pm.TruncatedNormal.dist(mu=mu2, sigma=sigma2, lower=0.0)

        mu3 = pm.Uniform('mu3',lower=0.1,upper=0.6)
        sigma3 = pm.Uniform('sigma3',lower=0.0001,upper=0.5)
        obs3 = pm.TruncatedNormal.dist(mu=mu3, sigma=sigma3, lower=0.0)

        w = pm.Dirichlet('w', a=np.array([1., .4, .4]))
        like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2, obs3], observed=ISI_data)

        step = pm.NUTS(target_accept=0.95)
        trace = pm.sample(step=step,draws=1000,chains=4,tune=1000,cores=4)
        ppc_trace = pm.sample_posterior_predictive(trace,model=model)
        #import matplotlib.pyplot as plt
        #pm.traceplot(trace)
        #az.plot_trace(trace)
        plt.show()

    map_estimate = pm.find_MAP(model=model)
    
    del map_estimate['w_simplex__']
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    del map_estimate['mu2_interval__']
    del map_estimate['sigma2_interval__']
    del map_estimate['mu3_interval__']
    del map_estimate['sigma3_interval__']
    
    map_estimate['w1'] = map_estimate['w'][0]
    map_estimate['w2'] = map_estimate['w'][1]
    map_estimate['w3'] = map_estimate['w'][2]

    del map_estimate['w']


    return map_estimate,trace,ppc_trace


In [ ]:
concatenated_dataframes = [elem.T for elem in tqdm(final)]
final_result = pd.concat(concatenated_dataframes, axis=0)
final_result

In [ ]:
print(file_name)
file_path = '/Users/Gaia_1/Desktop/tesi/Data after PP/'+file_name+'.csv'
final_result.to_csv(file_path, index=False)

In [ ]:
dataset = pd.read_csv('Data after PP/'+file_name+'.csv')
dataset

In [ ]:
dataset=final_result
dataset = dataset.reset_index(drop=True)
dataset

dataset = dataset.drop(['Unnamed: 0'],axis=1)

In [ ]:
dataset['Target'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score
X = dataset.drop(['Target','Stimulation'],axis=1)
y= dataset['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    stratify=y,
                                                    random_state=42)

In [ ]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
cv = StratifiedKFold(4)

In [ ]:
pca = PCA(n_components=5,svd_solver='full')
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)
print(pca.fit(X_train).explained_variance_ratio_,sum(pca.fit(X_train).explained_variance_ratio_))

# Tree

In [ ]:
params = {'max_depth':np.linspace(2,10,dtype='int'),'criterion':['gini','entropy']}

clf = DecisionTreeClassifier(random_state=12)

gs = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,n_jobs=-1)

gs=gs.fit(X_train,y_train)

gs_pca = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,n_jobs=-1)
gs_pca = gs_pca.fit(X_train_pca,y_train)

In [ ]:
print("Best parameters set found :",gs.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs.predict(X_test)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
print('ROC AUC: ',roc_auc)
print('\n\n Result with PCA\n')
print("Best parameters set found :",gs_pca.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs_pca.predict(X_test_pca)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
print('ROC AUC: ',roc_auc)

# RF

In [ ]:
params = {'n_estimators':np.linspace(10,150,dtype='int',num=10),'max_depth':np.linspace(2,5,dtype='int'),'criterion':['gini','entropy']}

clf = RandomForestClassifier(n_jobs=-1,random_state=12)

gs = GridSearchCV(clf, params, scoring='roc_auc',cv=cv,verbose=10,n_jobs=-1)

gs=gs.fit(X_train,y_train)

In [ ]:
print("Best parameters set found :",gs.best_params_,'\n')
print("Classification report on Test set\n")

y_true, y_pred = y_test, gs.predict(X_test)

accuracy = round(accuracy_score(y_true, y_pred),3)
recall = round(recall_score(y_true, y_pred),3)
precision = round(precision_score(y_true, y_pred),3)
roc_auc = round(roc_auc_score(y_true, y_pred),3)

print('Accuracy: ',accuracy)
print('Recall: ',recall)
print('Precision: ',precision)
print('ROC AUC: ',roc_auc)

In [ ]:
def this_poiproc(neurons,target,stim):
    from scipy.stats import ks_2samp
    dataframe = pd.DataFrame()
    #counter_net=1
    counter=0
    #for net in list_dir_ok:
    #print(counter_net,') ',net)
    #counter_net+=1
    list_neurons = neurons #np.genfromtxt(net, delimiter=',')
    counter=0
    print('Original number of neurons: ',len(list_neurons))
    for neuron in tqdm(list_neurons):
        neuron=neuron[neuron>0*10000]
        neuron=neuron[neuron<200*10000]
        print('  Neuron with ',neuron.shape[0],'spikes')
        if neuron.shape[0]>1000:
            
            counter+=1
        else:
            print('    Excluded neuron with n spikes = ',neuron.shape[0])
            continue
        
        ISI_healthy = np.diff(neuron)/10000
        map_estimate = Bayesian_mixture_model(ISI_healthy)
        map_estimate['Target']=int(target)
        map_estimate['Stimulation']=int(stim)
        df = pd.DataFrame.from_dict(map_estimate,orient='index')
        dataframe = pd.concat([dataframe,df],axis = 1)
    print('Final number of neurons: ',counter)
    print('Target = ',target)
    return dataframe



    #ks_2samp(lista_samples,ISI_healthy,mode = 'asymp')


In [ ]:
def this_spike_sorting(name_data,complete_string,threshold,clustering,coeff,c1):
    #file reading:
    data = h5py.File(complete_string,'r')
    data_readings = data['Data']['Recording_0']['AnalogStream']['Stream_0']['ChannelData'][()]
    info = data['Data']['Recording_0']['AnalogStream']['Stream_0']['InfoChannel'][()]
    info_table = pd.DataFrame(info, columns = list(info.dtype.fields.keys()))
    labels = info_table['Label']
    readings = pd.DataFrame(data = data_readings.transpose(), columns = labels)
    fs = 10000 #Sampling Frequency
    print('data shape: ',readings.shape)
    prova=readings.drop([b'Ref'],axis=1)
    #prova=prova.iloc[inizio:fine, :10]
    #prova=prova.iloc[:, :10]
    ref=readings[b'Ref']
    #ref=ref[inizio:fine]
    #filtering:
    prova_rows = range(prova.shape[0])
    filt_prova = pd.DataFrame(data = 0, columns=prova.columns, index=prova_rows, dtype = "float32")
    lowcut = 300
    highcut = 3000
    fs=10000
    order=8
    b,a=butter_bandpass(lowcut,highcut,fs,order=order)
    filt_ref=filtfilt(b,a,ref)
    for x in tqdm(range(prova.shape[1])):
        filt_prova.values[:,x] = scipy.signal.filtfilt(b, a, prova.values[:,x])
    for electrode in prova.columns:
        filt_prova[electrode] = filt_prova[electrode] - filt_ref
    prova=filt_prova
    #detection:
    if threshold==0:
        threshold=[]
        for i,electrode in enumerate(tqdm(prova.columns)):
            threshold.append(coeff*(scipy.stats.median_abs_deviation(prova[electrode].values,scale='normal')))            
    all_ind=[]
    for i,electrode in enumerate(tqdm(prova.columns)):
        channel=prova[electrode]
        thresh=threshold[i]
        ind=find_all_spikes(channel,thresh)
        all_ind.append(ind)
    #spike extraction:
    cut_outs=[]
    all_new=[]
    for i,electrode in enumerate(tqdm(prova.columns)):
        ind=all_ind[i]
        channel=prova[electrode]
        cut_outs1,all_new1=cut_all(ind,channel,c1)
        cut_outs.append(cut_outs1)
        all_new.append(all_new1)    
    # Clustering:
    final_data=[]
    if clustering=='kmeans':
            for channel in (tqdm(range(len(cut_outs)))):
                #channel_clusters1=comparative_clus(cut_outs[channel],all_new[channel],prova.iloc[:,channel])
                channel_clusters1=clus(cut_outs[channel],all_new[channel],prova.iloc[:,channel])
                final_data.append(channel_clusters1)
    elif clustering=='dbscan':
            for channel in (tqdm(range(len(cut_outs)))):
                eps=int(scipy.stats.median_abs_deviation(prova.iloc[:,channel])/2)
                channel_clusters1=dbscan_clustering(cut_outs[channel],all_new[channel],prova.iloc[:,channel],eps)
                final_data.append(channel_clusters1)
    neurons=[]
    for channel in final_data:
        for neuron in channel:
            neurons.append(neuron)
    print(len(neurons),' neurons detected and sorted')
    adj_neur=[]
    counter = 0
    max_len=0
    for neuron in neurons:
        print('counter: ',counter,neuron.shape[0])
        if neuron.shape[0]>max_len:
            max_len=neuron.shape[0]
        counter+=1
    for neuron in neurons:
        if neuron.shape[0]<max_len and neuron.shape[0]>=1000:
            diff = max_len-neuron.shape[0]
            adj_neur.append(np.concatenate((neuron,np.zeros([diff]))))
    save_data = 'After'+name_data+'.txt'
    np.savetxt("/Users/Gaia_1/Desktop/tesi/Data after SS prova/%s.txt" % save_data,adj_neur, delimiter=', ', fmt='%12.8f')
    print(save_data)
    return neurons
